In [ ]:
import gc
import os
import time
from datetime import datetime
import cPickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import log_loss

In [ ]:
def save_submission(submission, name='submission', path='.', with_dttm=False):
    suffix = '_' + datetime.now().strftime('%Y-%m-%d_%H-%M-%S') if with_dttm else ''
    filename = os.path.join(path, '%s%s.csv' % (name, suffix))
    submission.to_csv(filename, index=False)
    return filename

In [ ]:
train_df = pd.read_csv('../input/train.csv')
train_df = train_df.dropna().reset_index(drop=True)
train_df.head(n=10)

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.info()

In [ ]:
#test_df = pd.read_csv('data/test.csv')
#test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
#test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [ ]:
submission = pd.concat([test_df.test_id.astype(int), pd.DataFrame(predicted[:,1], 
                                        columns=['is_duplicate'], index=test_df.index)], axis=1)
print submission.shape
save_submission(submission, 'submission_BOW_chars', with_dttm=False)

In [ ]:
#models_and_test_matricies['BOW_chars_not_equal'] = (model_lr_bow_chars, X_BOW_chars)

In [ ]:
with open('models_and_test_matricies_2.pkl', 'wb') as fid:
    cPickle.dump(models_and_test_matricies, fid)

<h3>Извлечение признаков: Bag of words (word) </h3>

In [ ]:
del_punct_trans_table = dict((ord(char), None) for char in string.punctuation)

def nlp_preprocessor(sentence):
    return delete_punctuation_unicode(sentence.lower())

def nlp_tokenizer(sentence):
    tokens = word_tokenize(sentence)
    ll = WordNetLemmatizer()
    stemmer = SnowballStemmer(language='english', ignore_stopwords=True)
    tokens = [ll.lemmatize(item) for item in tokens]
    tokens = [stemmer.stem(item) for item in tokens]
    return tokens

def delete_punctuation_unicode(to_translate):
    return to_translate.translate(del_punct_trans_table)

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [ ]:
del BOW_q1, BOW_q2

In [ ]:
predicted = model_lr_BOW_words_not_equal.predict_proba(X_BOW_words_not_equal)

In [ ]:
#models_and_test_matricies['BOW_words_not_equal'] = (model_lr_BOW_words_not_equal, X_BOW_words_not_equal)

In [ ]:
#with open('models_and_test_matricies.pkl', 'wb') as fid:
    #cPickle.dump(models_and_test_matricies, fid)

In [ ]:
from sklearn.metrics.pairwise import paired_cosine_distances

In [ ]:
TFIDF = TfidfVectorizer(preprocessor=nlp_preprocessor, tokenizer=nlp_tokenizer, analyzer='word', max_df=0.999, min_df=20, 
                        norm='l2', use_idf=True, smooth_idf=True)
TFIDF.fit(pd.concat((train_df['question1'],train_df['question2'])).unique())

TFIDF_q1 = TFIDF.transform(train_df.question1)
TFIDF_q2 = TFIDF.transform(train_df.question2)

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [ ]:
X_TFIDF = paired_cosine_distances(TFIDF_q1,TFIDF_q2).reshape(-1, 1)
del TFIDF_q1, TFIDF_q2

In [ ]:
predicted = model_lr_tfidf.predict_proba(X_TFIDF)

In [ ]:
#models_and_test_matricies['TFIDF'] = (model_lr_tfidf, X_TFIDF)

<h3>Использование нескольких моделей: усреднение</h3>

In [ ]:
#subm_1 = pd.read_csv('./answers/1/submission_BOW_chars_2017-05-08_19-53-25.csv')
#subm_2 = pd.read_csv('./answers/1/submission_BOW_words_2017-05-09_06-02-19.csv')
#subm_mean = pd.DataFrame(subm_1)
#subm_mean['is_duplicate'] = (subm_1['is_duplicate'] + subm_2['is_duplicate']) / 2
#save_submission(subm_mean, 'submission_BOW_mean', with_dttm=True)